In [9]:
from utils import weights_dp, count_profit, count_weights, upper_bound
import random

In [3]:
def correct_chromosome(chromosome: list, weights: list, profits: list, max_weight: int):
    from random import randint
    one_indices = [i for i, item in enumerate(chromosome) if item == 1]
    cur_weight = count_weights(chromosome, weights)
    cur_profit = count_profit(chromosome, profits)
    while cur_weight > max_weight:
        cur_index = one_indices[randint(0, len(one_indices) - 1)]
        one_indices.remove(cur_index)
        chromosome[cur_index] = 0
        cur_weight -= weights[cur_index]
        cur_profit -= profits[cur_index]
    return chromosome, cur_profit, cur_weight

In [4]:
def generate_chromosome(capacity: int, profits: list, weights: list, ):
    from random import random
    chromosome = []
    for i in range(len(weights)):
        gene = 0 if random() < 0.5 else 1
        chromosome.append(gene)
    chromosome, profit, weight = correct_chromosome(chromosome, weights, profits, capacity)
    return chromosome, profit, weight

In [5]:
generate_chromosome(11, [10, 5, 7], [5, 6, 5])

([0, 1, 1], 12, 11)

In [48]:
def mutate_chromosome(chromosome: list):
    mut_index = random.randint(0, len(chromosome) - 1)
    chromosome[mut_index] = int(not chromosome[mut_index])
    return chromosome

In [271]:
N_GROUPS = 4
SELECT_INTERVALS = [0.1, 0.3, 0.6, 1]

def genetic_alg(cap: int, profits: list, weights: list, mutation_prob = 0.15, n_chromosomes = 8, n_iterations=30):
    chromosomes_list = []
    for i in range(n_chromosomes):
        chromosomes_list.append(generate_chromosome(cap, profits, weights))
    chromosomes_list = sorted(chromosomes_list, key=lambda item: item[1])
    
    for i in range(n_iterations):
        group1_index = upper_bound(SELECT_INTERVALS, random.random()) + 1
        chromo1_index = random.randint((group1_index - 1) * 4, group1_index * 4 - 1)
        
        group2_index = upper_bound(SELECT_INTERVALS, random.random()) + 1
        chromo2_index = random.randint((group1_index - 1) * 4, group1_index * 4 - 1)
        
        while chromo2_index == chromo1_index:
            group2_index = upper_bound(SELECT_INTERVALS, random.random()) + 1
            chromo2_index = random.randint((group1_index - 1) * 4, group1_index * 4 - 1)
            
        chromo1 = chromosomes_list[chromo1_index][0]
        chromo2 = chromosomes_list[chromo2_index][0]
        
        swap_index = random.randint(0, len(chromo1) - 1)
        part1_1, part2_1= chromo1[:swap_index], chromo2[:swap_index]
        
        chromo1 = part2_1 + chromo1[swap_index:]
        chromo2 = part1_1 + chromo2[swap_index:]
        
        if random.random() < mutation_prob:
            if random.random() < 0.5:
                chromo1 = mutate_chromosome(chromo1)
            else:
                chromo2 = mutate_chromosome(chromo2)
        
        chromo1 = correct_chromosome(chromo1, weights, profits, cap)
        chromo2 = correct_chromosome(chromo2, weights, profits, cap)
        
        # print(chromo1, chromo2)
        
        chromosomes_list[chromo1_index] = chromo1
        chromosomes_list[chromo2_index] = chromo2
        
        chromosomes_list = sorted(chromosomes_list, key=lambda item: item[1])
     
    return chromosomes_list[-1]
    
    

In [7]:
folder = '../Lab2/benchmarks/'
files_list = []
for i in range(1, 8):
    files_list.append((f'{folder}p0{i}_c.txt', f'{folder}p0{i}_w.txt', f'{folder}p0{i}_p.txt', f'{folder}p0{i}_s.txt'))

In [290]:
for i, sample in enumerate(files_list):
    with open(sample[0], 'r') as c, open(sample[1], 'r') as w, open(sample[2], 'r') as p, open(sample[3], 'r') as s:
        capacity = int(c.read())
        weights = list(map(int, w.read().split()))
        profits = list(map(int, p.read().split()))
        answer = list(map(int, s.read().split()))
        dp_items, dp_ans = weights_dp(cap=capacity, p=profits, w=weights)
        assert answer == dp_items
        print(f'capacity: {capacity}')
        print(f'max profit: {dp_ans}')
        print(f'optimal items: {answer}')
        alg_items, alg_profit, alg_weight = genetic_alg(capacity, profits, weights, n_chromosomes=32, n_iterations=2500) #best: 32 chromes, 60 iters
        print('----------------------')
        print(f'genetic alg:')
        print(f'items: {alg_items}')
        print(f'profit: {alg_profit}')
        print(f'weight: {alg_items}')
        print('----------------------\n')
        

capacity: 165
max profit: 309
optimal items: [1, 1, 1, 1, 0, 1, 0, 0, 0, 0]
----------------------
genetic alg:
items: [1, 1, 1, 1, 0, 1, 0, 0, 0, 0]
profit: 309
weight: [1, 1, 1, 1, 0, 1, 0, 0, 0, 0]
----------------------



capacity: 26
max profit: 51
optimal items: [0, 1, 1, 1, 0]
----------------------
genetic alg:
items: [0, 1, 1, 1, 0]
profit: 51
weight: [0, 1, 1, 1, 0]
----------------------



capacity: 190
max profit: 150
optimal items: [1, 1, 0, 0, 1, 0]
----------------------
genetic alg:
items: [1, 1, 0, 0, 1, 0]
profit: 150
weight: [1, 1, 0, 0, 1, 0]
----------------------



capacity: 50
max profit: 107
optimal items: [1, 0, 0, 1, 0, 0, 0]
----------------------
genetic alg:
items: [1, 0, 0, 1, 0, 0, 0]
profit: 107
weight: [1, 0, 0, 1, 0, 0, 0]
----------------------



capacity: 104
max profit: 900
optimal items: [1, 0, 1, 1, 1, 0, 1, 1]
----------------------
genetic alg:
items: [1, 0, 1, 1, 1, 0, 1, 1]
profit: 900
weight: [1, 0, 1, 1, 1, 0, 1, 1]
---------------------